In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
from xgboost import XGBClassifier
import shap

# 1. CHARGEMENT DU FICHIER DÉJÀ TRAITÉ
# Attention : Assure-toi que le chemin est bon
df = pd.read_csv('../data/processed/train_processed.csv')

print("✅ Données traitées chargées !")
print(df.info()) # Vérifie que tout est bien numérique (float/int)

# 2. SÉPARATION CIBLE / FEATURES
target_col = 'heart_disease'

# Si la cible est encore du texte (ex: "Malade"/"Sain"), on convertit
if df[target_col].dtype == 'object':
    df[target_col] = df[target_col].apply(lambda x: 1 if x in ['Malade', 'Yes', '1'] else 0)

X = df.drop(columns=[target_col])
y = df[target_col]

# 3. SPLIT TRAIN / TEST
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# 4. ENTRAÎNEMENT (Plus besoin de Pipeline de nettoyage !)
print("\n🚀 Démarrage de l'entraînement XGBoost sur données pré-traitées...")

model = XGBClassifier(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=5,
    eval_metric='logloss',
    random_state=42
)

model.fit(X_train, y_train)
print("✅ Modèle entraîné !")

# 5. ÉVALUATION
y_pred = model.predict(X_test)
y_prob = model.predict_proba(X_test)[:, 1]

print("\n--- RÉSULTATS ---")
print(classification_report(y_test, y_pred))

auc_score = roc_auc_score(y_test, y_prob)
print(f"🎯 SCORE AUC : {auc_score:.4f}")

# 6. SHAP (Interprétabilité)
# C'est plus simple ici car on n'a pas de pipeline complexe
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_test)

plt.figure()
shap.summary_plot(shap_values, X_test, plot_type="bar")
plt.show()